## Creation of City Dimension Table 

In [2]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.City (
    CityKey INT NOT NULL,
    WWICityID INT NOT NULL,
    City STRING NOT NULL,
    StateProvince STRING NOT NULL,
    Country STRING NOT NULL,
    Continent STRING NOT NULL,
    SalesTerritory STRING NOT NULL,
    Region STRING NOT NULL,
    Subregion STRING NOT NULL,
    Location STRING NOT NULL,
    LatestRecordedPopulation BIGINT NOT NULL
    )

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 3, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql
-- Creation of temp view and inserting the data into city dimension table
CREATE OR REPLACE TEMP VIEW v_city 
AS  
SELECT 
    c.CityID, 
    c.CityName, 
    s.StateProvinceName, 
    co.CountryName, 
    co.Continent, 
    s.SalesTerritory, 
    co.Region, 
    co.Subregion,
    c.Location, 
    COALESCE(c.LatestRecordedPopulation, 0) AS Population
FROM application_cities AS c
INNER JOIN application_stateprovinces AS s
    ON c.StateProvinceID = s.StateProvinceID
INNER JOIN application_countries AS co
    ON s.CountryID = co.CountryID;

INSERT INTO dimension.City
select ROW_NUMBER() OVER (ORDER BY NULL) AS CityKey,
    CityID AS WWICityID, 
    CityName AS City,
    StateProvinceName AS StateProvince,
    CountryName AS Country, 
    Continent, 
    SalesTerritory, 
    Region, 
    Subregion,
    Location,
    Population AS LatestRecordedPopulation
 from v_city


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Customer Dimension Table

In [5]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.Customer (
    CustomerKey INT NOT NULL,
    WWICustomerID INT NOT NULL,
    Customer STRING NOT NULL,
    BillToCustomer STRING NOT NULL,
    Category STRING NOT NULL,
    BuyingGroup STRING NOT NULL,
    PrimaryContact STRING NOT NULL,
    PostalCode STRING NOT NULL
    )


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 7, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [6]:
%%sql
-- Creation of temp view and inserting the data into customer dimension table
CREATE OR REPLACE TEMP VIEW v_customer
AS 
 SELECT c.CustomerID, c.CustomerName, bt.CustomerName as billToCustomer, cc.CustomerCategoryName,
               bg.BuyingGroupName, p.FullName, c.DeliveryPostalCode
        FROM sales_customers  AS c
        INNER JOIN sales_buyinggroups  AS bg
        ON c.BuyingGroupID = bg.BuyingGroupID
        INNER JOIN sales_customercategories AS cc
        ON c.CustomerCategoryID = cc.CustomerCategoryID
        INNER JOIN sales_customers AS bt
        ON c.BillToCustomerID = bt.CustomerID
        INNER JOIN application_people AS p
        ON c.PrimaryContactPersonID = p.PersonID;

INSERT INTO dimension.Customer
select ROW_NUMBER() OVER (ORDER BY null) AS CustomerKey,
    CustomerID AS WWICustomerID, 
    CustomerName AS City,
    billToCustomer AS BillToCustomer,
    CustomerCategoryName AS Category, 
    BuyingGroupName AS BuyingGroup, 
    FullName AS PrimaryContact, 
    DeliveryPostalCode AS PostalCode
 from v_customer;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Date Dimension Table 

In [8]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.Date (
    Date DATE,
    DayNumber INT,
    Day STRING,
    Month INT,
    ShortMonth STRING,
    CalendarMonthNumber INT,
    CalendarMonthLabel STRING,
    CalendarYear INT,
    CalendarYearLabel STRING,
    FiscalMonthNumber INT,
    FiscalMonthLabel STRING,
    FiscalYear INT,
    FiscalYearLabel STRING,
    ISOWeekNumber INT
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql
-- Generate a range of dates
WITH DateRange AS (
    SELECT
        SEQUENCE(
            TO_DATE('2013-01-01'),  -- MinDate
            TO_DATE('2016-12-31'),  -- MaxDate
            INTERVAL 1 DAY          -- Increment by 1 day
        ) AS Dates
),
 
-- Explode the range into individual rows
ExplodedDates AS (
    SELECT EXPLODE(Dates) AS Date FROM DateRange
),
 
-- Add date attributes
DateAttributes AS (
    SELECT
        Date,
        DAYOFMONTH(Date) AS DayNumber,
        DATE_FORMAT(Date, 'EEEE') AS Day,             -- Full weekday name
        MONTH(Date) AS Month,
        DATE_FORMAT(Date, 'MMM') AS ShortMonth,      -- Abbreviated month name
        MONTH(Date) AS CalendarMonthNumber,
        CONCAT(YEAR(Date), '-', LPAD(MONTH(Date), 2, '0')) AS CalendarMonthLabel,
        YEAR(Date) AS CalendarYear,
        CAST(YEAR(Date) AS STRING) AS CalendarYearLabel,
        MONTH(Date) AS FiscalMonthNumber,           -- Assuming fiscal months align with calendar months
        CONCAT('FY', YEAR(Date), '-', LPAD(MONTH(Date), 2, '0')) AS FiscalMonthLabel,
        YEAR(Date) AS FiscalYear,                   -- Assuming fiscal year aligns with calendar year
        CONCAT('FY', YEAR(Date)) AS FiscalYearLabel,
        WEEKOFYEAR(Date) AS ISOWeekNumber           -- ISO week number
    FROM ExplodedDates
)
 
-- Insert the generated data into the table
INSERT INTO dimension.Date
SELECT *
FROM DateAttributes;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Employee Dimension Table

In [11]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.Employee (
    EmployeeKey INT NOT NULL,
    WWIEmployeeID INT NOT NULL,
    Employee STRING NOT NULL,
    PreferredName STRING NOT NULL,
    IsSalesperson BOOLEAN NOT NULL
    )
   

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 14, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
%%sql
-- Creation of temp view and inserting the data into employee dimension table
CREATE OR REPLACE TEMP VIEW v_employee
AS 
  SELECT p.PersonID, p.FullName, p.PreferredName, p.IsSalesperson, p.Photo
        FROM application_people AS p;
        
INSERT INTO dimension.Employee
select ROW_NUMBER() OVER (ORDER BY null) AS EmployeeKey,
    PersonID AS WWIEmployeeID, 
    FullName AS Employee,
    PreferredName AS PreferredName,
    IsSalesperson AS IsSalesperson
 
 from v_employee;


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of payment method Dimension Table

In [14]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.PaymentMethod (
    PaymentMethodKey INT NOT NULL,
    WWIPaymentMethodID INT NOT NULL,
    PaymentMethod STRING NOT NULL
    )

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
%%sql
-- Creation of temp view and inserting the data into payment_method dimension table
CREATE OR REPLACE TEMP VIEW v_paymentMethod
AS 
SELECT p.PaymentMethodID, p.PaymentMethodName
        FROM application_paymentmethods AS p;

INSERT INTO dimension.PaymentMethod
select ROW_NUMBER() OVER (ORDER BY NULL) AS PaymentMethodKey,
    PaymentMethodID AS WWIPaymentMethodID,
    PaymentMethodName AS PaymentMethod
 from v_paymentMethod;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of StockItem Dimension Table

In [17]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.StockItem(
    StockItemKey INT NOT NULL,
    WWIStockItemID INT NOT NULL,
    StockItem STRING NOT NULL,
    Color STRING , 
    SellingPackage STRING NOT NULL,
    BuyingPackage STRING NOT NULL,
    Brand STRING ,
    Size STRING ,
    LeadTimeDays INT NOT NULL,
    QuantityPerOuter INT NOT NULL,
    IsChillerStock Boolean NOT NULL,
    Barcode STRING ,
    TaxRate DECIMAL(18,3) NOT NULL,
    UnitPrice DECIMAL(18,2) NOT NULL,
    RecommendedRetailPrice DECIMAL(18,2),
    TypicalWeightPerUnit DECIMAL(18,3) NOT NULL
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [18]:
%%sql
-- Creation of temp view and inserting the data into payment_method dimension table
CREATE OR REPLACE TEMP VIEW v_stock_item
AS 
   SELECT si.StockItemID, si.StockItemName, c.ColorName, spt.PackageTypeName,
               bpt.PackageTypeName as BuyingPackageTypeName, si.Brand, si.Size, si.LeadTimeDays, si.QuantityPerOuter,
               si.IsChillerStock, si.Barcode, si.TaxRate, si.UnitPrice, si.RecommendedRetailPrice,
               si.TypicalWeightPerUnit, si.Photo
        FROM warehouse_stockitems AS si
        INNER JOIN warehouse_packagetypes  AS spt
        ON si.UnitPackageID = spt.PackageTypeID
        INNER JOIN warehouse_packagetypes  AS bpt
        ON si.OuterPackageID = bpt.PackageTypeID
        LEFT OUTER JOIN warehouse_colors  AS c
        ON si.ColorID = c.ColorID;
INSERT INTO dimension.StockItem
select ROW_NUMBER() OVER (ORDER BY NULL) AS StockItemKey,
    StockItemID AS WWIStockItemID,
    StockItemName AS StockItem,
    ColorName AS Color,
    PackageTypeName AS SellingPackage,
    BuyingPackageTypeName AS BuyingPackage,
    Brand AS Brand,
    Size AS Size,
    LeadTimeDays AS LeadTimeDays,
    QuantityPerOuter AS QuantityPerOuter,
    IsChillerStock AS IsChillerStock,
    Barcode AS Barcode, 
    TaxRate AS TaxRate,
    UnitPrice AS UnitPrice,
    RecommendedRetailPrice AS RecommendedRetailPrice,
    TypicalWeightPerUnit AS TypicalWeightPerUnit

 from v_stock_item;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 24, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of supplier Dimension Table 

In [20]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.Supplier (
    SupplierKey INT NOT NULL,
    WWISupplierID INT NOT NULL,
    Supplier STRING NOT NULL,
    Category STRING NOT NULL,
    PrimaryContact STRING NOT NULL,
    SupplierReference STRING NOT NULL,
    PaymentDays INT NOT NULL,
    PostalCode STRING NOT NULL
    )

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 26, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [21]:
%%sql
-- Creation of temp view and inserting the data into supplier dimension table
CREATE OR REPLACE TEMP VIEW v_supplier
AS 
   SELECT s.SupplierID, s.SupplierName, sc.SupplierCategoryName, p.FullName, s.SupplierReference,
               s.PaymentDays, s.DeliveryPostalCode
        FROM purchasing_suppliers AS s
        INNER JOIN purchasing_suppliercategories sc
        ON s.SupplierCategoryID = sc.SupplierCategoryID
        INNER JOIN application_people  p
        ON s.PrimaryContactPersonID = p.PersonID;
INSERT INTO dimension.Supplier
select ROW_NUMBER() OVER (ORDER BY NULL) AS SupplierKey,
    SupplierID AS WWISupplierID,
    SupplierName AS Supplier,
    SupplierCategoryName AS Category,
    FullName AS PrimaryContact,
    SupplierReference AS SupplierReference,
    PaymentDays AS PaymentDays,
    DeliveryPostalCode AS PostalCode
 from v_supplier;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 28, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Transaction Type Dimension Table 

In [23]:
%%sql
CREATE TABLE IF NOT EXISTS dimension.TransactionType (
    TransactionTypeKey INT NOT NULL,
    WWITransactionTypeID INT NOT NULL,
    TransactionType STRING NOT NULL
    )

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 30, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [24]:
%%sql
-- Creation of temp view and inserting the data into transaction type dimension table
CREATE OR REPLACE TEMP VIEW v_transaction_type
AS 
  SELECT p.TransactionTypeID, p.TransactionTypeName
        FROM application_transactiontypes AS p;
INSERT INTO dimension.TransactionType
select ROW_NUMBER() OVER (ORDER BY NULL) AS TransactionTypeKey,
    TransactionTypeID AS WWITransactionTypeID,
    TransactionTypeName AS TransactionType
 from v_transaction_type;


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 32, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

## Creation of stock holding Fact Table 

In [26]:
%%sql
CREATE TABLE IF NOT EXISTS fact.StockHolding (
    StockHoldingKey BIGINT NOT NULL, 
    StockItemKey INT NOT NULL,
    QuantityOnHand INT NOT NULL,
    BinLocation STRING NOT NULL,
    LastStocktakeQuantity INT NOT NULL,
    LastCostPrice DECIMAL(18,2) NOT NULL,
    ReorderLevel INT NOT NULL,
    TargetStockLevel INT NOT NULL
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 34, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [27]:
%%sql
-- storing the stored procedure into temp view v_stock_holding
CREATE OR REPLACE TEMP VIEW v_stock_holding AS
  SELECT sih.QuantityOnHand AS QuantityOnHand,
           sih.BinLocation AS BinLocation,
           sih.LastStocktakeQuantity AS LastStocktakeQuantity,
           sih.LastCostPrice AS LastCostPrice,
           sih.ReorderLevel AS ReorderLevel,
           sih.TargetStockLevel AS TargetStockLevel,
           sih.StockItemID AS WWIStockItemID
    FROM warehouse_stockitemholdings AS sih
    ORDER BY sih.StockItemID;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 35, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [28]:
%%sql
-- joining with dimension.stock_item to get StockItemKey from it and inserting data into
--  fact.stock_holding 
INSERT INTO fact.StockHolding
SELECT ROW_NUMBER() OVER (ORDER BY NULL) AS StockHoldingKey,
       ds.StockItemKey as StockItemKey,
       QuantityOnHand, 
       BinLocation,
       LastStocktakeQuantity,
       LastCostPrice,
       ReorderLevel,
       TargetStockLevel
FROM v_stock_holding 
INNER JOIN dimension.StockItem AS ds
ON v_stock_holding.WWIStockItemID = ds.WWIStockItemID

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 36, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of order Fact Table 

In [30]:
%%sql
CREATE TABLE IF NOT EXISTS fact.Order (
    OrderKey BIGINT NOT NULL,
    CityKey INT NOT NULL,
    CustomerKey INT NOT NULL,
    StockItemKey INT NOT NULL,
    OrderDateKey DATE NOT NULL,
    PickedDateKey DATE,
    SalespersonKey INT NOT NULL,
    PickerKey INT,
    WWIOrderID INT NOT NULL,
    WWIBackorderID INT,
    Description STRING NOT NULL,
    Package STRING NOT NULL,
    Quantity INT NOT NULL,
    UnitPrice DECIMAL(18,2) NOT NULL,
    TaxRate DECIMAL(18,3) NOT NULL,
    TotalExcludingTax DECIMAL(30,2) NOT NULL,
    TaxAmount DECIMAL(35,2) NOT NULL,
    TotalIncludingTax DECIMAL(36,2) NOT NULL
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 38, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [31]:
%%sql
 CREATE OR REPLACE TEMP VIEW v_order AS
 SELECT CAST(o.OrderDate AS date) AS OrderDateKey,
           CAST(ol.PickingCompletedWhen AS date) AS PickedDateKey,
           o.OrderID AS WWIOrderID,
           o.BackorderOrderID AS WWIBackorderID,
           ol.Description,
           pt.PackageTypeName AS Package,
           ol.Quantity AS Quantity,
           ol.UnitPrice AS UnitPrice,
           ol.TaxRate AS TaxRate,
           ROUND(ol.Quantity * ol.UnitPrice, 2) AS TotalExcludingTax,
           ROUND(ol.Quantity * ol.UnitPrice * ol.TaxRate / 100.0, 2) AS TaxAmount,
           ROUND(ol.Quantity * ol.UnitPrice, 2) + ROUND(ol.Quantity * ol.UnitPrice * ol.TaxRate / 100.0, 2) AS TotalIncludingTax,
           c.DeliveryCityID AS WWICityID,
           c.CustomerID AS WWICustomerID,
           ol.StockItemID AS WWIStockItemID,
           o.SalespersonPersonID AS WWISalespersonID,
           o.PickedByPersonID AS WWIPickerID,
           CASE WHEN ol.LastEditedWhen > o.LastEditedWhen THEN ol.LastEditedWhen ELSE o.LastEditedWhen END AS LastModifiedWhen
    FROM sales_orders AS o
    INNER JOIN sales_orderlines AS ol
    ON o.OrderID = ol.OrderID
    INNER JOIN warehouse_packagetypes AS pt
    ON ol.PackageTypeID = pt.PackageTypeID
    INNER JOIN sales_customers AS c
    ON c.CustomerID = o.CustomerID
    ORDER BY o.OrderID;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 39, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [32]:
%%sql
INSERT INTO fact.Order
SELECT ROW_NUMBER() OVER(ORDER BY NULL) AS OrderKey, 
      dc.CityKey AS CityKey,
      dcust.CustomerKey AS CustomerKey,
      si.StockItemKey AS StockItemKey,
      OrderDateKey, 
      PickedDateKey,
      emp1.EmployeeKey AS SalesPersonKey, 
      emp2.EmployeeKey AS PickerKey,
      v_order.WWIOrderID AS WWIOrderID,
      v_order.WWIBackorderID AS WWIBackorderID,
      v_order.Description AS Description,
      v_order.Package AS Package,
      v_order.Quantity AS Quantity,
      v_order.UnitPrice AS UnitPrice,
      v_order.TaxRate AS TaxRate,
      v_order.TotalExcludingTax AS TotalExcludingTax,
      v_order.TaxAmount AS TaxAmount,
      v_order.TotalIncludingTax AS TotalIncludingTax
FROM v_order
INNER JOIN dimension.City AS dc
ON v_order.WWICityID = dc.WWICityID
INNER JOIN dimension.Customer AS dcust
ON v_order.WWICustomerID = dcust.WWICustomerID
INNER JOIN dimension.StockItem AS si
ON v_order.WWIStockItemID = si.WWIStockItemID
INNER JOIN dimension.Employee AS emp1
ON v_order.WWISalespersonID = emp1.WWIEmployeeID
INNER JOIN dimension.Employee AS emp2
ON v_order.WWIPickerID = emp2.WWIEmployeeID



StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 40, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Purchase Fact Table 

In [34]:
%%sql

CREATE TABLE IF NOT EXISTS fact.Purchase( 
    PurchaseKey BIGINT NOT NULL,
    DateKey DATE NOT NULL,
    SupplierKey INT NOT NULL,
    StockItemKey INT NOT NULL,
    WWIPurchaseOrderID INT,
    OrderedOuters INT NOT NULL,
    OrderedQuantity INT NOT NULL,
    ReceivedOuters INT NOT NULL,
    Package STRING NOT NULL,
    IsOrderFinalized BOOLEAN NOT NULL
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 42, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [35]:
%%sql
CREATE OR REPLACE TEMP VIEW v_purchase
AS
SELECT CAST(po.OrderDate AS date) AS DateKey,
       po.PurchaseOrderID AS WWIPurchaseOrderID,
       pol.OrderedOuters AS OrderedOuters,
       pol.OrderedOuters * si.QuantityPerOuter AS OrderedQuantity,
       pol.ReceivedOuters AS ReceivedOuters,
       pt.PackageTypeName AS Package,
       pol.IsOrderLineFinalized AS IsOrderFinalized,
       po.SupplierID AS WWISupplierID,
       pol.StockItemID AS WWIStockItemID,
       CASE WHEN pol.LastEditedWhen > po.LastEditedWhen THEN pol.LastEditedWhen ELSE po.LastEditedWhen END AS LastModifiedWhen
FROM Purchasing_PurchaseOrders AS po
INNER JOIN Purchasing_PurchaseOrderLines AS pol
ON po.PurchaseOrderID = pol.PurchaseOrderID
INNER JOIN Warehouse_StockItems AS si
ON pol.StockItemID = si.StockItemID
INNER JOIN Warehouse_PackageTypes AS pt
ON pol.PackageTypeID = pt.PackageTypeID
ORDER BY po.PurchaseOrderID;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 43, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [36]:
%%sql
INSERT INTO fact.Purchase
SELECT ROW_NUMBER() OVER(ORDER BY NULL) AS PurchaseKey, 
   DateKey,
   ds.SupplierKey AS SupplierKey,
   si.StockItemKey AS StockItemKey, 
   WWIPurchaseOrderID, 
   OrderedOuters, 
   OrderedQuantity,
   ReceivedOuters, 
   Package,
   IsOrderFinalized
   
FROM v_purchase   
INNER JOIN dimension.supplier AS ds
ON v_purchase.WWISupplierID = ds.WWISupplierID
INNER JOIN dimension.StockItem AS si
WHERE v_purchase.WWIStockItemID = si.WWIStockItemID;


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 44, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of Sale Fact Table 

In [38]:
%%sql
CREATE TABLE IF NOT EXISTS fact.Sale (
    SaleKey BIGINT NOT NULL,
    CityKey INT NOT NULL,
    CustomerKey INT ,
    BillToCustomerKey INT ,
    StockItemKey INT ,
    InvoiceDateKey DATE ,
    DeliveryDateKey DATE,
    SalespersonKey INT ,
    WWIInvoiceID INT ,
    Description STRING ,
    Package STRING ,
    Quantity INT ,
    UnitPrice DECIMAL(18, 2) ,
    TaxRate DECIMAL(5, 2) ,
    TotalExcludingTax DECIMAL(18, 2) ,
    TaxAmount DECIMAL(18, 2) ,
    Profit DECIMAL(18, 2) ,
    TotalIncludingTax DECIMAL(18, 2) ,
    TotalDryItems INT ,
    TotalChillerItems INT 
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 46, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [39]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW v_sale
AS
SELECT 
    CAST(i.InvoiceDate AS date) AS InvoiceDateKey,
    CAST(i.ConfirmedDeliveryTime AS date) AS DeliveryDateKey,
    i.InvoiceID AS WWIInvoiceID,
    il.Description,
    pt.PackageTypeName AS Package,
    il.Quantity,
    il.UnitPrice AS UnitPrice,
    il.TaxRate AS TaxRate,
    il.ExtendedPrice - il.TaxAmount AS TotalExcludingTax,
    il.TaxAmount AS TaxAmount,
    il.LineProfit AS Profit,
    il.ExtendedPrice AS TotalIncludingTax,
    CASE WHEN si.IsChillerStock = 0 THEN il.Quantity ELSE 0 END AS TotalDryItems,
    CASE WHEN si.IsChillerStock <> 0 THEN il.Quantity ELSE 0 END AS TotalChillerItems,
    c.DeliveryCityID AS WWICityID,
    i.CustomerID AS WWICustomerID,
    i.BillToCustomerID AS WWIBillToCustomerID,
    il.StockItemID AS WWIStockItemID,
    i.SalespersonPersonID AS WWISalespersonID,
    CASE 
        WHEN il.LastEditedWhen > i.LastEditedWhen THEN il.LastEditedWhen 
        ELSE i.LastEditedWhen 
    END AS LastModifiedWhen
FROM Sales_Invoices AS i
INNER JOIN Sales_InvoiceLines AS il
    ON i.InvoiceID = il.InvoiceID
INNER JOIN Warehouse_StockItems AS si
    ON il.StockItemID = si.StockItemID
INNER JOIN Warehouse_PackageTypes AS pt
    ON il.PackageTypeID = pt.PackageTypeID
INNER JOIN Sales_Customers AS c
    ON i.CustomerID = c.CustomerID
INNER JOIN Sales_Customers AS bt
    ON i.BillToCustomerID = bt.CustomerID;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 47, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [40]:
%%sql
INSERT INTO fact.Sale
SELECT ROW_NUMBER() OVER(ORDER BY NULL) AS SaleKey,
       dc.CityKey,
       dc1.CustomerKey, 
       dc2.CustomerKey AS BillToCustomerKey,
       si.stockItemKey,
       InvoiceDateKey,
       DeliveryDateKey, 
       emp.employeeKey AS SalesPersonKey, 
       WWIInvoiceID,
       Description,
       Package,
       Quantity,
       v_sale.Unitprice,
       v_sale.TaxRate,
       TotalExcludingTax,
       TaxAmount,
       Profit,
       TotalIncludingTax, 
       TotalDryItems, 
       TotalChillerItems   
FROM v_sale 
LEFT JOIN dimension.City AS dc
ON v_sale.WWICityID = dc.WWICityID
LEFT JOIN dimension.Customer AS dc1
ON v_sale.WWICustomerID = dc1.WWICustomerID
LEFT JOIN dimension.Customer AS dc2
ON v_sale.WWIBillToCustomerID = dc2.WWICustomerID 
LEFT JOIN dimension.StockItem AS si
ON  v_sale.WWIStockItemID = si.WWIStockItemID 
LEFT JOIN dimension.Employee AS emp
ON  v_sale.WWISalespersonID = emp.WWIEmployeeID

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 48, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of movement Fact Table 

In [43]:
%%sql
CREATE TABLE IF NOT EXISTS fact.Movement (
    MovementKey BIGINT NOT NULL,
    DateKey DATE NOT NULL,
    StockItemKey INT NOT NULL,
    CustomerKey INT,
    SupplierKey INT,
    TransactionTypeKey INT NOT NULL,
    WWIStockItemTransactionID INT NOT NULL,
    WWIInvoiceID INT,
    WWIPurchaseOrderID INT,
    Quantity INT NOT NULL
);

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 51, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [44]:
%%sql
CREATE TABLE staging.MovementStaging (
    MovementStagingKey BIGINT,
    DateKey DATE,
    StockItemKey INT,
    CustomerKey INT,
    SupplierKey INT,
    TransactionTypeKey INT,
    WWIStockItemTransactionID INT,
    WWIInvoiceID INT,
    WWIPurchaseOrderID INT,
    Quantity INT,
    WWIStockItemID INT,
    WWICustomerID INT,
    WWISupplierID INT,
    WWITransactionTypeID INT,
    LastModifiedWhen TIMESTAMP
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 52, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [45]:
%%sql
CREATE OR REPLACE TEMP VIEW v_movement
AS
SELECT CAST(sit.TransactionOccurredWhen AS date) AS DateKey,
       sit.StockItemTransactionID AS WWIStockItemTransactionID,
       sit.InvoiceID AS WWIInvoiceID,
       sit.PurchaseOrderID AS WWIPurchaseOrderID,
       CAST(sit.Quantity AS int) AS Quantity,
       sit.StockItemID AS WWIStockItemID,
       sit.CustomerID AS WWICustomerID,
       sit.SupplierID AS WWISupplierID,
       sit.TransactionTypeID AS WWITransactionTypeID,
       sit.TransactionOccurredWhen AS TransactionOccurredWhen
FROM warehouse_stockitemtransactions AS sit
ORDER BY sit.StockItemTransactionID;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 53, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [46]:
%%sql
INSERT INTO staging.MovementStaging(DateKey, WWIStockItemTransactionID, WWIInvoiceID, WWIPurchaseOrderID,Quantity,
WWIStockItemID, WWICustomerID, WWISupplierID, WWITransactionTypeID, LastModifiedWhen)
SELECT DateKey,
       WWIStockItemTransactionID,
       WWIInvoiceID,
       WWIPurchaseOrderID,
       Quantity,
       WWIStockItemID,
       WWICustomerID,
       WWISupplierID,
       WWITransactionTypeID,
       TransactionOccurredWhen
FROM v_movement;

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 54, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [47]:
%%sql
MERGE INTO staging.MovementStaging AS m
USING dimension.TransactionType AS tt
ON m.WWITransactionTypeID = tt.WWITransactionTypeID
WHEN MATCHED THEN
UPDATE SET m.TransactionTypeKey = COALESCE(tt.TransactionTypeKey, 0);

MERGE INTO staging.MovementStaging AS m
USING dimension.StockItem AS si
ON m.WWIstockItemID = si.WWIStockItemID
WHEN MATCHED THEN
UPDATE SET m.stockItemKey = si.stockItemKey;

MERGE INTO staging.MovementStaging AS m
USING dimension.Customer AS cu
ON m.WWICustomerID = cu.WWICustomerID
WHEN MATCHED THEN
UPDATE SET m.CustomerKey = cu.CustomerKey;

MERGE INTO staging.MovementStaging AS m
USING dimension.Customer AS c2
ON m.WWICustomerID = c2.WWICustomerID
WHEN MATCHED THEN
UPDATE SET m.CustomerKey = COALESCE(c2.CustomerKey, 0);

MERGE INTO staging.MovementStaging AS m
USING dimension.Supplier AS s
ON m.WWISupplierID = s.WWISupplierID
WHEN MATCHED THEN
UPDATE SET m.SupplierKey = COALESCE(s.SupplierKey, 0);

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 59, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

In [48]:
%%sql
INSERT INTO fact.Movement
        (
          MovementKey,
         DateKey,
          StockItemKey,
           CustomerKey,
        SupplierKey,
             TransactionTypeKey,
         WWIStockItemTransactionID,
          WWIInvoiceID, 
          WWIPurchaseOrderID, 
          Quantity 
          )
SELECT 
  ROW_NUMBER() OVER(ORDER BY NULL) AS MovementKey,
  DateKey,
   StockItemKey,
   CustomerKey,
     SupplierKey,
      TransactionTypeKey,
        WWIStockItemTransactionID,
         WWIInvoiceID,
          WWIPurchaseOrderID,
           Quantity
FROM staging.MovementStaging

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 60, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

## Creation of transaction Fact Table

In [51]:
%%sql
CREATE TABLE IF NOT EXISTS fact.Transaction (
    TransactionKey BIGINT NOT NULL,
    DateKey DATE NOT NULL,
    CustomerKey INT,
    BillToCustomerKey INT,
    SupplierKey INT,
    TransactionTypeKey INT NOT NULL,
    PaymentMethodKey INT,
    WWICustomerTransactionID INT,
    WWISupplierTransactionID INT,
    WWIInvoiceID INT,
    WWIPurchaseOrderID INT,
    SupplierInvoiceNumber STRING,
    TotalExcludingTax DECIMAL(18, 2) NOT NULL,
    TaxAmount DECIMAL(18, 2) NOT NULL,
    TotalIncludingTax DECIMAL(18, 2) NOT NULL,
    OutstandingBalance DECIMAL(18, 2) NOT NULL,
    IsFinalized BOOLEAN NOT NULL
);

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 63, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [52]:
%%sql
CREATE TABLE staging.TransactionStaging (
    TransactionStagingKey BIGINT,
    DateKey DATE,
    CustomerKey INT,
    BillToCustomerKey INT,
    SupplierKey INT,
    TransactionTypeKey INT,
    PaymentMethodKey INT,
    WWICustomerTransactionID INT,
    WWISupplierTransactionID INT,
    WWIInvoiceID INT,
    WWIPurchaseOrderID INT,
    SupplierInvoiceNumber STRING,
    TotalExcludingTax DECIMAL(9, 2),
    TaxAmount DECIMAL(9, 2),
    TotalIncludingTax DECIMAL(9, 2),
    OutstandingBalance DECIMAL(9, 2),
    IsFinalized BOOLEAN,
    WWICustomerID INT,
    WWIBillToCustomerID INT,
    WWISupplierID INT,
    WWITransactionTypeID INT,
    WWIPaymentMethodID INT,
    LastModifiedWhen TIMESTAMP
)

StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 64, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [53]:
%%sql
CREATE OR REPLACE TEMP VIEW v_transaction
AS
SELECT CAST(ct.TransactionDate AS date) AS DateKey,
       ct.CustomerTransactionID AS WWICustomerTransactionID,
       CAST(NULL AS int) AS WWISupplierTransactionID,
       ct.InvoiceID AS WWIInvoiceID,
       CAST(NULL AS int) AS WWIPurchaseOrderID,
       CAST(NULL AS varchar(200)) AS SupplierInvoiceNumber,
       ct.AmountExcludingTax AS TotalExcludingTax,
       ct.TaxAmount AS TaxAmount,
       ct.TransactionAmount AS TotalIncludingTax,
       ct.OutstandingBalance AS OutstandingBalance,
       ct.IsFinalized AS IsFinalized,
       COALESCE(i.CustomerID, ct.CustomerID) AS WWICustomerID,
       ct.CustomerID AS WWIBillToCustomerID,
       CAST(NULL AS int) AS WWISupplierID,
       ct.TransactionTypeID AS WWITransactionTypeID,
       ct.PaymentMethodID AS WWIPaymentMethodID,
       ct.LastEditedWhen AS LastModifiedWhen
FROM Sales_CustomerTransactions AS ct
LEFT OUTER JOIN Sales_Invoices AS i
ON ct.InvoiceID = i.InvoiceID


UNION ALL

SELECT CAST(st.TransactionDate AS date) AS DateKey,
       CAST(NULL AS int) AS WWICustomerTransactionID,
       st.SupplierTransactionID AS WWISupplierTransactionID,
       CAST(NULL AS int) AS WWIInvoiceID,
       st.PurchaseOrderID AS WWIPurchaseOrderID,
       st.SupplierInvoiceNumber AS SupplierInvoiceNumber,
       st.AmountExcludingTax AS TotalExcludingTax,
       st.TaxAmount AS TaxAmount,
       st.TransactionAmount AS TotalIncludingTax,
       st.OutstandingBalance AS OutstandingBalance,
       st.IsFinalized AS IsFinalized,
       CAST(NULL AS int) AS WWICustomerID,
       CAST(NULL AS int) AS WWIBillToCustomerID,
       st.SupplierID AS WWISupplierID,
       st.TransactionTypeID AS WWITransactionTypeID,
       st.PaymentMethodID AS WWIPaymentMethodID,
       st.LastEditedWhen AS LastModifiedWhen
FROM Purchasing_SupplierTransactions AS st;


INSERT INTO staging.TransactionStaging
(
DateKey,
WWICustomerTransactionID,
WWISupplierTransactionID,
WWIInvoiceID,
WWIPurchaseOrderID,
SupplierInvoiceNumber,
TotalExcludingTax,
TaxAmount,
TotalIncludingTax,
OutstandingBalance,
IsFinalized,
WWICustomerID,
WWIBillToCustomerID,
WWISupplierID,
WWITransactionTypeID,
WWIPaymentMethodID,
LastModifiedWhen

)
SELECT 
DateKey,
WWICustomerTransactionID,
WWISupplierTransactionID,
WWIInvoiceID,
WWIPurchaseOrderID,
SupplierInvoiceNumber,
TotalExcludingTax,
TaxAmount,
TotalIncludingTax,
OutstandingBalance,
IsFinalized,
WWICustomerID,
WWIBillToCustomerID,
WWISupplierID,
WWITransactionTypeID,
WWIPaymentMethodID,
LastModifiedWhen

 FROM 
v_transaction;



-- Merging data into staging.transaction for adding keys to the table

MERGE INTO staging.TransactionStaging AS t
USING dimension.Customer AS c
ON t.WWICustomerID = c.WWICustomerID
WHEN MATCHED THEN
UPDATE SET t.CustomerKey = COALESCE(c.CustomerKey, 0);


MERGE INTO staging.TransactionStaging AS t
USING dimension.Customer AS c
ON t.WWIBillToCustomerID = c.WWICustomerID
WHEN MATCHED THEN
UPDATE SET t.BillToCustomerKey = COALESCE(c.CustomerKey, 0);


MERGE INTO staging.TransactionStaging AS t
USING dimension.Supplier AS s
ON t.WWISupplierID = s.WWISupplierID
WHEN MATCHED THEN
UPDATE SET t.SupplierKey = COALESCE(s.SupplierKey, 0);

MERGE INTO staging.TransactionStaging AS t
USING dimension.TransactionType AS tt
ON t.WWITransactionTypeID = tt.WWITransactionTypeID
WHEN MATCHED THEN
UPDATE SET t.TransactionTypeKey = COALESCE(tt.TransactionTypeKey, 0);


MERGE INTO staging.TransactionStaging AS t
USING dimension.PaymentMethod AS pm
ON t.WWIPaymentMethodID = pm.WWIPaymentMethodID
WHEN MATCHED THEN
UPDATE SET t.PaymentMethodKey = COALESCE(pm.PaymentMethodKey, 0);






INSERT INTO fact.Transaction
        (TransactionKey, DateKey, CustomerKey, BillToCustomerKey, SupplierKey, TransactionTypeKey,
         PaymentMethodKey, WWICustomerTransactionID, WWISupplierTransactionID,
         WWIInvoiceID, WWIPurchaseOrderID, SupplierInvoiceNumber, TotalExcludingTax,
         TaxAmount, TotalIncludingTax, OutstandingBalance, IsFinalized)
SELECT ROW_NUMBER() OVER(ORDER BY NULL) AS TransactionKey,
        DateKey,
        CustomerKey,
        BillToCustomerKey,
        SupplierKey,
        TransactionTypeKey,
        PaymentMethodKey,
        WWICustomerTransactionID,
        WWISupplierTransactionID,
        WWIInvoiceID,
        WWIPurchaseOrderID,
        SupplierInvoiceNumber,
        TotalExcludingTax,
        TaxAmount,
        TotalIncludingTax,
        OutstandingBalance,
        IsFinalized 
FROM staging.TransactionStaging
;


StatementMeta(, 4aba1b06-7b89-4a57-89c3-1c3e2ea61674, 72, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 1 rows and 4 fields>

<Spark SQL result set with 0 rows and 0 fields>